In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/npy-1-5sec/y_train_task1.npy
/kaggle/input/npy-1-5sec/x_val_task2.npy
/kaggle/input/npy-1-5sec/y_train_task4.npy
/kaggle/input/npy-1-5sec/y_test_task1.npy
/kaggle/input/npy-1-5sec/x_test_task4.npy
/kaggle/input/npy-1-5sec/y_val_task4.npy
/kaggle/input/npy-1-5sec/y_val_task1.npy
/kaggle/input/npy-1-5sec/x_train_task4.npy
/kaggle/input/npy-1-5sec/x_val_task4.npy
/kaggle/input/npy-1-5sec/x_test_task1.npy
/kaggle/input/npy-1-5sec/x_val_task1.npy
/kaggle/input/npy-1-5sec/y_test_task2.npy
/kaggle/input/npy-1-5sec/y_train_task2.npy
/kaggle/input/npy-1-5sec/y_test_task4.npy
/kaggle/input/npy-1-5sec/x_test_task2.npy
/kaggle/input/npy-1-5sec/x_train_task1.npy
/kaggle/input/npy-1-5sec/x_train_task2.npy
/kaggle/input/npy-1-5sec/y_val_task2.npy
/kaggle/input/npy-task/y_train_task1.npy
/kaggle/input/npy-task/x_val_task2.npy
/kaggle/input/npy-task/y_train_task4.npy
/kaggle/input/npy-task/y_test_task1.npy
/kaggle/input/npy-task/x_test_task4.npy
/kaggle/input/npy-task/y_val_task4.npy
/kag

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict
from typing import Tuple, Union
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from copy import deepcopy
#from google.colab import drive
#import glob
import os
import pandas as pd
from tensorflow import keras

fp = '/kaggle/input/npy-1-5sec/'
%cd /kaggle/input/npy-1-5sec/

/kaggle/input/npy-1-5sec


In [3]:
all_features = ['','_all','_256','_1024','_all_imu_no_stairs','_all_keypoints_no_stairs','_all_no_stairs'][0]
tasks = ['task1','task2', 'task4']
# Initialize empty lists to store the arrays for each task
x_train_list = []
x_val_list = []
x_test_list = []
y_train_list = []
y_val_list = []
y_test_list = []

for task in tasks:
  x_train_task = np.load(fp+'x_train_'+all_features+task+'.npy')
  y_train_task = np.load(fp+'y_train_'+all_features+task+'.npy')
  x_val_task = np.load(fp+'x_val_'+all_features+task+'.npy')
  y_val_task = np.load(fp+'y_val_'+all_features+task+'.npy')
  x_test_task = np.load(fp+'x_test_'+all_features+task+'.npy')
  y_test_task = np.load(fp+'y_test_'+all_features+task+'.npy')

  # Append the current data arrays to their respective lists
  x_train_list.append(x_train_task)
  x_val_list.append(x_val_task)
  x_test_list.append(x_test_task)
  y_train_list.append(y_train_task)
  y_val_list.append(y_val_task)
  y_test_list.append(y_test_task)

# Concatenate all arrays into single arrays
x_train = np.concatenate(x_train_list, axis=0)
x_val = np.concatenate(x_val_list, axis=0)
x_test = np.concatenate(x_test_list, axis=0)
y_train = np.concatenate(y_train_list, axis=0)
y_val = np.concatenate(y_val_list, axis=0)
y_test = np.concatenate(y_test_list, axis=0)

# Shuffle the data indices
indices = np.arange(len(x_train))
np.random.shuffle(indices)

# Select 30% of the data
num_samples = int(0.3 * len(x_train))
selected_indices = indices[:num_samples]

# Select the corresponding data and labels
x_train = x_train[selected_indices]
y_train = y_train[selected_indices]

# Create a mask for y_train not equal to 1
mask = y_train != 1

# Apply the mask to filter out y_train == 1 and corresponding x_train
x_train = x_train[mask]
y_train = y_train[mask]

# Shuffle the data indices
indices = np.arange(len(x_val))
np.random.shuffle(indices)

# Select 30% of the data
num_samples = int(0.3 * len(x_val))
selected_indices = indices[:num_samples]

# Select the corresponding data and labels
x_val = x_val[selected_indices]
y_val = y_val[selected_indices]

# Create a mask for y_train not equal to 1
mask = y_val != 1

# Apply the mask to filter out y_train == 1 and corresponding x_train
x_val = x_val[mask]
y_val = y_val[mask]

# Shuffle the data indices
indices = np.arange(len(x_test))
np.random.shuffle(indices)

# Select 30% of the data
num_samples = int(0.3 * len(x_test))
selected_indices = indices[:num_samples]

# Select the corresponding data and labels
x_test = x_test[selected_indices]
y_test = y_test[selected_indices]

# Create a mask for y_train not equal to 1
mask = y_test != 1

# Apply the mask to filter out y_train == 1 and corresponding x_train
x_test = x_test[mask]
y_test = y_test[mask]

# Adjust the labels
y_train = y_train - 2
y_val = y_val - 2
y_test = y_test - 2

print('Train shape: ', x_train.shape, 'Train Label shape: ', y_train.shape, '\n', 'Val Shape: ', x_val.shape, 'Val Label shape: ', y_val.shape, '\n', 'Test shape: ', x_test.shape, 'Test Label shape: ', y_test.shape)

Train shape:  (1218, 3000, 54) Train Label shape:  (1218,) 
 Val Shape:  (410, 3000, 54) Val Label shape:  (410,) 
 Test shape:  (410, 3000, 54) Test Label shape:  (410,)


In [4]:
print(np.unique(y_train))

[0. 1. 2. 3. 4. 5. 6.]


In [5]:
input_dir = '/kaggle/input/base_model'
for dirname, _, filenames in os.walk(input_dir):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten

model = keras.models.Sequential([
    keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(3000, 54)),
    keras.layers.MaxPooling1D(pool_size=11),
    keras.layers.LSTM(50, input_shape=(3000, 54)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(7, activation='softmax')

])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
#model.summary()

# Train the model
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=32, epochs=10)

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

# Make predictions on new data
predictions = model.predict(x_test)
#model.save('few_model.h5')


Epoch 1/10
39/39 [==============================] - 11s 197ms/step - loss: 1.1964 - accuracy: 0.6396 - val_loss: 0.8043 - val_accuracy: 0.7220
Epoch 2/10
39/39 [==============================] - 6s 160ms/step - loss: 0.6921 - accuracy: 0.7553 - val_loss: 0.6407 - val_accuracy: 0.7902
Epoch 3/10
39/39 [==============================] - 6s 162ms/step - loss: 0.6145 - accuracy: 0.7775 - val_loss: 0.6151 - val_accuracy: 0.7829
Epoch 4/10
39/39 [==============================] - 6s 160ms/step - loss: 0.5957 - accuracy: 0.7783 - val_loss: 0.5906 - val_accuracy: 0.7829
Epoch 5/10
39/39 [==============================] - 6s 166ms/step - loss: 0.5826 - accuracy: 0.7783 - val_loss: 0.5862 - val_accuracy: 0.7732
Epoch 6/10
39/39 [==============================] - 6s 159ms/step - loss: 0.5815 - accuracy: 0.7709 - val_loss: 0.5768 - val_accuracy: 0.7829
Epoch 7/10
39/39 [==============================] - 6s 162ms/step - loss: 0.5723 - accuracy: 0.7775 - val_loss: 0.5765 - val_accuracy: 0.7732
Epoch

In [7]:
model.save('/kaggle/working/my_model.h5')


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
import os
os.listdir('/kaggle/working')

['my_model.h5', '.virtual_documents']

In [9]:
predictions = model.predict(x_test)
print(predictions)

13/13 [==============================] - 1s 49ms/step
[[9.98437166e-01 6.86592830e-04 6.55310170e-04 ... 4.91013452e-05
  1.26627318e-04 1.55631842e-05]
 [6.40206635e-01 1.46242529e-01 2.10331678e-01 ... 9.10275092e-04
  9.46998538e-04 6.66763284e-04]
 [7.08317637e-01 1.16447113e-01 1.71471506e-01 ... 1.22347614e-03
  9.16028977e-04 7.49951578e-04]
 ...
 [9.94425535e-01 2.31869519e-03 2.98127136e-03 ... 7.40628966e-05
  1.38908334e-04 2.66921797e-05]
 [9.93384182e-01 3.08538741e-03 3.32449703e-03 ... 3.94411254e-05
  1.21879915e-04 1.65242218e-05]
 [6.34371638e-01 1.24420442e-01 2.32737795e-01 ... 2.87918886e-03
  1.60322769e-03 2.32368242e-03]]


In [10]:
pred_class=np.argmax(predictions, axis=1)
print(pred_class)

[0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0
 4 0 0 0 4 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 4 0 0 0 0 0 4 0 0 0 0 4 0
 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 4 0 0 0 0 4 0 0 0 0 0 0 0 4 0 0 0
 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 4 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 4 4 4 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 4 0 0 0 0 0
 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 4 0 0 0 0 0 4 0 0 0 4
 0 0 0 0 0 0 0 4 0 4 0 0 4 0 0 0 0 0 0 4 0 0 0 0 0 4 0 0 0 0 0 0 4 0 4 4 0
 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0
 4 0 0 0 0 0 4 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 4 0 4
 0 4 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 4 0 0 0 0 4 0 0 0 0 0 0 4 0 0 0 0 4 0
 0 0 0]


In [11]:
x_test_task1 = np.load(fp+'x_test_task1.npy')
y_test_task1 = np.load(fp+'y_test_task1.npy')
x_test_task2 = np.load(fp+'x_test_task2.npy')
y_test_task2 = np.load(fp+'y_test_task2.npy')
x_test_task4 = np.load(fp+'x_test_task4.npy')
y_test_task4 = np.load(fp+'y_test_task4.npy')

# Create a mask for y_train not equal to 1
mask = y_test_task1 != 1

# Apply the mask to filter out y_train == 1 and corresponding x_train
x_test_task1 = x_test_task1[mask]
y_test_task1 = y_test_task1[mask]

# Create a mask for y_train not equal to 1
mask = y_test_task2 != 1

# Apply the mask to filter out y_train == 1 and corresponding x_train
x_test_task2 = x_test_task2[mask]
y_test_task2 = y_test_task2[mask]

# Create a mask for y_train not equal to 1
mask = y_test_task4 != 1

# Apply the mask to filter out y_train == 1 and corresponding x_train
x_test_task4 = x_test_task4[mask]
y_test_task4 = y_test_task4[mask]

In [12]:
y_test_task1 = y_test_task1 - 2
y_test_task2 = y_test_task2 - 2
y_test_task4 = y_test_task4 - 2

In [13]:
pred_task1 = model.predict(x_test_task1)
print(pred_task1)

18/18 [==============================] - 1s 46ms/step
[[9.9749446e-01 1.1247054e-03 1.0861300e-03 ... 6.1626743e-05
  1.7972493e-04 1.4751538e-05]
 [9.9764025e-01 1.0761785e-03 1.0311746e-03 ... 6.6428969e-05
  1.3289093e-04 1.8426384e-05]
 [9.9444240e-01 1.9277050e-03 3.0286433e-03 ... 2.1144538e-04
  2.6536599e-04 3.5083558e-05]
 ...
 [9.9773377e-01 1.1021383e-03 1.0563413e-03 ... 1.4989980e-05
  7.2002433e-05 6.6190164e-06]
 [9.9553055e-01 2.0816470e-03 1.8634120e-03 ... 8.8695000e-05
  3.4345506e-04 3.5247140e-05]
 [9.9670744e-01 1.6865988e-03 1.3288385e-03 ... 6.4989421e-05
  1.5645334e-04 2.3780860e-05]]


In [14]:
y_test_task1

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [15]:
# Assuming pred_task1 is your predictions array
# Slice to get the first two elements of each subarray
task1_classes = pred_task1[:, :1]

# Find the maximum of the first two elements in each subarray
pred_task1_BIM = np.argmax(task1_classes, axis=1)

print(pred_task1_BIM)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [16]:
len(y_test_task1)

552

In [17]:
# Assuming max_of_first_two and y_test_task1 are your arrays
# Perform element-wise comparison
not_equal = pred_task1_BIM != y_test_task1

# Count the number of non-equal elements
count_not_equal_1 = np.sum(not_equal)
accuracy_task1=1-count_not_equal_1/len(y_test_task1)
print('count_not_equal_1:', count_not_equal_1)
print('accuracy for task1 with BIM:',accuracy_task1)

count_not_equal_1: 0
accuracy for task1 with BIM: 1.0


In [18]:
pred_task2 = model.predict(x_test_task2)
print(pred_task2)

21/21 [==============================] - 1s 48ms/step
[[0.6195101  0.1560345  0.22081867 ... 0.00102902 0.00094086 0.00077824]
 [0.64105374 0.14594285 0.20977214 ... 0.00091354 0.00095133 0.00066884]
 [0.63933194 0.14655164 0.21090955 ... 0.00090692 0.00094255 0.00066462]
 ...
 [0.6413563  0.14583586 0.20957239 ... 0.0009147  0.00095288 0.00066959]
 [0.63933295 0.14655109 0.21090902 ... 0.00090693 0.00094256 0.00066463]
 [0.6398861  0.14635578 0.21054332 ... 0.00090904 0.00094537 0.00066598]]


In [19]:
np.argmax(pred_task2, axis=1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [20]:
# Assuming pred_task1 is your predictions array
# Slice to get the first two elements of each subarray
task2_classes = pred_task2[:, :3]

# Find the maximum of the first two elements in each subarray
pred_task2_BIM = np.argmax(task2_classes, axis=1)

print(pred_task2_BIM)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [21]:
print(y_test_task2)

[2. 0. 0. 0. 0. 0. 2. 2. 0. 1. 0. 0. 0. 2. 0. 0. 0. 2. 0. 0. 0. 2. 0. 0.
 2. 0. 0. 0. 0. 0. 0. 0. 0. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 2. 0. 0.
 0. 0. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 2. 0. 0. 0. 0. 2. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 2. 0. 0. 0. 0. 2. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 0.
 2. 0. 2. 0. 0. 1. 2. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0.
 0. 2. 0. 0. 0. 0. 2. 0. 2. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 2. 0. 0. 2.
 0. 0. 2. 0. 0. 0. 2. 0. 2. 0. 1. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 2. 0. 2. 2. 2. 2. 0. 2. 0. 0. 0. 2. 2. 0. 0. 0. 0. 0. 2. 0. 1.
 2. 0. 0. 1. 1. 0. 1. 2. 1. 2. 2. 0. 0. 0. 2. 0. 2. 0. 1. 0. 2. 1. 0. 0.
 0. 0. 1. 2. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 2. 0. 2. 0. 0. 0. 0.
 0. 0. 0. 2. 0. 0. 0. 0. 2. 2. 2. 0. 0. 0. 0. 0. 1. 2. 0. 0. 0. 2. 1. 0.
 0. 2. 0. 0. 0. 1. 0. 0. 0. 0. 2. 2. 1. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0.
 2. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 2. 0. 0. 0. 0. 0. 0. 2. 1. 2. 0.
 0. 0. 2. 1. 1. 2. 1. 2. 1. 0. 0. 0. 0. 0. 0. 0. 2.

In [22]:
# Assuming max_of_first_two and y_test_task1 are your arrays
# Perform element-wise comparison
not_equal = pred_task2_BIM != y_test_task2

# Count the number of non-equal elements
count_not_equal_2 = np.sum(not_equal)
accuracy_task2=1-count_not_equal_2/len(y_test_task2)
print('count_not_equal_2:',count_not_equal_2)
print('accuracy for task2 with BIM:',accuracy_task2)

count_not_equal_2: 225
accuracy for task2 with BIM: 0.6621621621621622


In [23]:
pred_task4 = model.predict(x_test_task4)
print(pred_task4)

6/6 [==============================] - 0s 41ms/step
[[0.00941977 0.00569139 0.0096628  ... 0.39143252 0.02521639 0.28674862]
 [0.0094197  0.00569134 0.00966274 ... 0.39143145 0.02521645 0.28675032]
 [0.01073631 0.00498545 0.00997704 ... 0.39670157 0.02833056 0.24293414]
 ...
 [0.01436026 0.00645652 0.01097448 ... 0.42607132 0.02918018 0.2375455 ]
 [0.00957957 0.00617119 0.01041982 ... 0.38656324 0.02529556 0.2850523 ]
 [0.00640835 0.00159849 0.00603002 ... 0.49760965 0.01831545 0.31511158]]


In [24]:
np.argmax(pred_task2, axis=1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [25]:
# Assuming pred_task1 is your predictions array
# Slice to get the first two elements of each subarray
task4_classes = pred_task4[:, 4:8]

# Find the maximum of the first two elements in each subarray
pred_task4_BIM = np.argmax(task4_classes, axis=1)
pred_task4_BIM = pred_task4_BIM + 4
print(pred_task4_BIM)

[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]


In [26]:
y_test_task4

array([4., 6., 4., 3., 4., 5., 4., 6., 6., 4., 4., 6., 6., 6., 4., 4., 4.,
       3., 6., 6., 4., 3., 6., 3., 4., 6., 4., 4., 3., 3., 4., 6., 4., 3.,
       6., 6., 4., 4., 6., 4., 4., 3., 4., 6., 6., 3., 4., 3., 6., 3., 3.,
       4., 3., 6., 4., 5., 6., 4., 4., 3., 4., 4., 3., 6., 4., 4., 4., 4.,
       4., 6., 4., 4., 6., 3., 4., 6., 3., 4., 6., 4., 3., 6., 3., 6., 3.,
       4., 4., 3., 4., 4., 5., 3., 6., 3., 3., 6., 6., 4., 6., 6., 3., 6.,
       4., 4., 3., 3., 4., 3., 5., 6., 3., 6., 4., 3., 4., 3., 6., 4., 6.,
       4., 6., 6., 3., 6., 6., 4., 5., 4., 4., 4., 6., 3., 3., 4., 4., 3.,
       4., 4., 4., 4., 6., 3., 3., 6., 6., 3., 3., 5., 3., 3., 6., 6., 4.,
       3., 4., 6., 4., 6., 3., 6., 6., 6., 4., 5., 4., 6., 4.])

In [27]:
# Assuming max_of_first_two and y_test_task1 are your arrays
# Perform element-wise comparison
not_equal = pred_task4_BIM != y_test_task4

# Count the number of non-equal elements
count_not_equal_4 = np.sum(not_equal)
accuracy_task4=1-count_not_equal_4/len(y_test_task4)
print('count_not_equal_4:',count_not_equal_4)
print('accuracy for task4 with BIM:',accuracy_task4)

count_not_equal_4: 102
accuracy for task4 with BIM: 0.3892215568862275


In [28]:
accuracy_all=1-(count_not_equal_1+count_not_equal_2+count_not_equal_4)/(len(y_test_task1)+len(y_test_task2)+len(y_test_task4))
print(accuracy_all)

0.763898916967509
